In [1]:
# ===================== Added L-VaR ACROSS METHODS (1d & 10d) =====================
# - Historical (empirical)
# - Variance–Covariance / Normal (parametric)
# - Monte Carlo (MVN)
# Liquidity add-on consistently applied across methods.
# ============================================================================
import pandas as pd
import numpy as np
from scipy.stats import norm

# -------------------- Parameters --------------------
CL = 0.99
z = 2.3263478740408408          # Normal quantile for CL
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2]) # A, B, C
H10 = 10                            # 10-day horizon length
ADDON_SCALING_FOR_10D = "constant"  # "constant" or "sqrt10"
RNG = np.random.default_rng(12345)  # for MC

# -------------------- Load returns --------------------
returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)
assets = list(returns.columns)
assert len(assets) == len(weights) == 3, "Expect exactly three assets aligned with weights."

# -------------------- Liquidity inputs --------------------
# Edit these with your live inputs; TTL is a liquidity (liquidation) horizon, not the risk horizon.
halfspread_bps = np.array([10.0, 15.0, 8.0])  # A, B, C
impact_bps     = np.array([ 5.0,  8.0, 4.0])  # A, B, C
TTL_Days       = np.array([ 3.0,  3.0, 3.0])  # per-asset TTL (days)

# -------------------- Liquidity add-on (per asset & totals) --------------------
# AddOn_component_i = Value_i * (bps/1e4) * sqrt(TTL_i)
values = total_value * weights
spread_add = values * (halfspread_bps/1e4) * np.sqrt(TTL_Days)
impact_add = values * (impact_bps/1e4)     * np.sqrt(TTL_Days)
liq_add_total_1d = float(spread_add.sum() + impact_add.sum())
liq_add_total_10d = liq_add_total_1d * (np.sqrt(H10) if ADDON_SCALING_FOR_10D.lower() == "sqrt10" else 1.0)

# -------------------- Helpers --------------------
def var_es_empirical(ret_series: np.ndarray, cl: float = CL, notional: float = total_value):
    """VaR/ES from empirical losses (Historical). ret_series is 1D returns."""
    losses = -ret_series * notional
    var = float(np.percentile(losses, cl * 100))
    es = float(losses[losses >= var].mean())
    return var, es

def var_es_normal_from_sigma(sigma: float):
    """Parametric VaR/ES from sigma (1-day)."""
    k = norm.pdf(z) / (1 - CL)  # ES multiplier
    var_1d = z * sigma * total_value
    es_1d  = k * sigma * total_value
    return var_1d, es_1d

def liq_adjust(v1d, v10d, add1d=liq_add_total_1d, add10d=liq_add_total_10d):
    """Apply liquidity add-on to 1d/10d metrics."""
    return v1d + add1d, v10d + add10d

# =============================================================================
# 1) HISTORICAL (EMPIRICAL)
# =============================================================================
# Per-asset (using each column separately)
hist_rows = []
for col in assets:
    var1d, es1d = var_es_empirical(returns[col].values)
    var10d, es10d = var1d*np.sqrt(H10), es1d*np.sqrt(H10)  # simple √time scaling for assets
    lvar1d, lvar10d = liq_adjust(var1d, var10d)
    les1d,  les10d  = liq_adjust(es1d,  es10d)
    hist_rows.append([col, var1d, es1d, var10d, es10d, lvar1d, les1d, lvar10d, les10d])

# Portfolio (empirical, from weighted returns)
port_ret = returns.values @ weights
p_var1d, p_es1d = var_es_empirical(port_ret)
p_var10d, p_es10d = p_var1d*np.sqrt(H10), p_es1d*np.sqrt(H10)
p_lvar1d, p_lvar10d = liq_adjust(p_var1d, p_var10d)
p_les1d,  p_les10d  = liq_adjust(p_es1d,  p_es10d)
hist_rows.append(["Portfolio", p_var1d, p_es1d, p_var10d, p_es10d, p_lvar1d, p_les1d, p_lvar10d, p_les10d])

hist_df = pd.DataFrame(hist_rows,
    columns=["Name","VaR_1d","ES_1d","VaR_10d","ES_10d","LVaR_1d","LES_1d","LVaR_10d","LES_10d"]
).set_index("Name")

# =============================================================================
# 2) VARIANCE–COVARIANCE / NORMAL (PARAMETRIC)
# =============================================================================
norm_rows = []
# Per-asset (using each asset's sigma)
sigma_assets = returns.std(ddof=1)
for col, sig in sigma_assets.items():
    var1d, es1d = var_es_normal_from_sigma(sig)
    var10d, es10d = var1d*np.sqrt(H10), es1d*np.sqrt(H10)
    lvar1d, lvar10d = liq_adjust(var1d, var10d)
    les1d,  les10d  = liq_adjust(es1d,  es10d)
    norm_rows.append([col, var1d, es1d, var10d, es10d, lvar1d, les1d, lvar10d, les10d])

# Portfolio (sigma from w^T Σ w)
cov = returns.cov().values
sigma_p = float(np.sqrt(weights @ cov @ weights.T))
p_var1d, p_es1d = var_es_normal_from_sigma(sigma_p)
p_var10d, p_es10d = p_var1d*np.sqrt(H10), p_es1d*np.sqrt(H10)
p_lvar1d, p_lvar10d = liq_adjust(p_var1d, p_var10d)
p_les1d,  p_les10d  = liq_adjust(p_es1d,  p_es10d)
norm_rows.append(["Portfolio", p_var1d, p_es1d, p_var10d, p_es10d, p_lvar1d, p_les1d, p_lvar10d, p_les10d])

norm_df = pd.DataFrame(norm_rows,
    columns=["Name","VaR_1d","ES_1d","VaR_10d","ES_10d","LVaR_1d","LES_1d","LVaR_10d","LES_10d"]
).set_index("Name")

# =============================================================================
# 3) MONTE CARLO (MVN CALIBRATED TO MEAN/COV)
# =============================================================================
mc_rows = []
mu = returns.mean().values
Sigma = returns.cov().values

# 1-day simulations
N1 = 40_000
ret_1d = RNG.multivariate_normal(mean=mu, cov=Sigma, size=N1)
loss_1d_assets = -ret_1d * total_value
port_loss_1d = - (ret_1d @ weights) * total_value

def var_es_from_losses(losses, cl=CL):
    v = float(np.percentile(losses, cl*100))
    e = float(losses[losses >= v].mean())
    return v, e

# per-asset 1d
v1d_assets, e1d_assets = [], []
for j in range(3):
    v, e = var_es_from_losses(loss_1d_assets[:, j])
    v1d_assets.append(v); e1d_assets.append(e)
# portfolio 1d
v1d_port, e1d_port = var_es_from_losses(port_loss_1d)

# 10-day path simulations (sum of 10 daily vectors)
N10 = 40_000
paths = RNG.multivariate_normal(mean=mu, cov=Sigma, size=(N10, H10))  # (N10,10,3)
ret_10d = paths.sum(axis=1)                                           # (N10,3)
loss_10d_assets = -ret_10d * total_value
port_loss_10d = - (ret_10d @ weights) * total_value

# per-asset 10d
v10d_assets, e10d_assets = [], []
for j in range(3):
    v, e = var_es_from_losses(loss_10d_assets[:, j])
    v10d_assets.append(v); e10d_assets.append(e)
# portfolio 10d
v10d_port, e10d_port = var_es_from_losses(port_loss_10d)

# assemble per-asset rows
for i, name in enumerate(assets):
    var1d, es1d = v1d_assets[i], e1d_assets[i]
    var10d, es10d = v10d_assets[i], e10d_assets[i]
    lvar1d, lvar10d = liq_adjust(var1d, var10d)
    les1d,  les10d  = liq_adjust(es1d,  es10d)
    mc_rows.append([name, var1d, es1d, var10d, es10d, lvar1d, les1d, lvar10d, les10d])

# add portfolio row
var1d, es1d = v1d_port, e1d_port
var10d, es10d = v10d_port, e10d_port
lvar1d, lvar10d = liq_adjust(var1d, var10d)
les1d,  les10d  = liq_adjust(es1d,  es10d)
mc_rows.append(["Portfolio", var1d, es1d, var10d, es10d, lvar1d, les1d, lvar10d, les10d])

mc_df = pd.DataFrame(mc_rows,
    columns=["Name","VaR_1d","ES_1d","VaR_10d","ES_10d","LVaR_1d","LES_1d","LVaR_10d","LES_10d"]
).set_index("Name")

# =============================================================================
# Save per-method results and a combined comparison table
# =============================================================================
hist_df.round(2).to_csv("sample_data/var_es_lvar_historical.csv")
norm_df.round(2).to_csv("sample_data/var_es_lvar_normal.csv")
mc_df.round(2).to_csv("sample_data/var_es_lvar_montecarlo.csv")

# Combined long-form table for quick comparison
def to_long(df, method_name):
    tmp = df.reset_index().melt(id_vars="Name", var_name="Metric", value_name="Value")
    tmp.insert(0, "Method", method_name)
    return tmp

combined = pd.concat([
    to_long(hist_df, "Historical"),
    to_long(norm_df, "Normal"),
    to_long(mc_df,   "Monte Carlo"),
], ignore_index=True)

combined.round(2).to_csv("sample_data/var_es_lvar_all_methods.csv", index=False)

# Optional: pretty pivot for console
pivot = combined.pivot_table(index=["Method","Name"], columns="Metric", values="Value")
print(pivot.round(2))
print("\nFiles written:\n",
      "- sample_data/var_es_lvar_historical.csv\n",
      "- sample_data/var_es_lvar_normal.csv\n",
      "- sample_data/var_es_lvar_montecarlo.csv\n",
      "- sample_data/var_es_lvar_all_methods.csv")


Metric                         ES_10d     ES_1d    LES_10d    LES_1d  \
Method      Name                                                       
Historical  Asset_A_Return   89138.45  28188.05   92048.30  31097.90   
            Asset_B_Return   57654.83  18232.06   60564.67  21141.90   
            Asset_C_Return  152564.97  48245.28  155474.82  51155.13   
            Portfolio        64376.59  20357.67   67286.44  23267.51   
Monte Carlo Asset_A_Return   81842.76  29885.65   84752.61  32795.50   
            Asset_B_Return   58863.00  19234.20   61772.84  22144.05   
            Asset_C_Return  144478.70  49859.22  147388.55  52769.06   
            Portfolio        59108.34  21622.58   62018.19  24532.42   
Normal      Asset_A_Return   99926.05  31599.39  102835.90  34509.24   
            Asset_B_Return   62344.01  19714.91   65253.85  22624.75   
            Asset_C_Return  164047.95  51876.52  166957.80  54786.36   
            Portfolio        73135.71  23127.54   76045.55  2603